# Automação Web e Busca de Informações com Python
 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: Produtos.xlsx

Para isso, vamos criar uma automação web:

- Usado as Bibliotecas pandas e selenium
- Importante: baixar o webdriver

In [32]:
#!pip install selenium
# - webdriver - chrome -> chromedriver
# - download webdriver chrome - https://chromedriver.chromium.org/downloads
# - instalado na pasta onde o JupyterNotebook(anaconda3) está instalado

In [33]:
from selenium import webdriver # permite criar o navegador
from selenium.webdriver.common.keys import Keys # permite você escrever no navegador
from selenium.webdriver.common.by import By # permite você selecionar intens do navegador 

# passo 1: Pegar a cotação do dolar
# - abrir o navegador
navegador = webdriver.Chrome()

# - entrar no google
navegador.get("https://www.google.com/")

# - pesquisar no google "cotação do dolar"
navegador.find_element('xpath',
'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação do dolar') 
navegador.find_element('xpath',
'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# - pegar a cotação que está no google
cotacao_dolar = navegador.find_element('xpath',
'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_dolar)

# passo 2: Pegar a cotação do euro
navegador.get("https://www.google.com/")
navegador.find_element('xpath',
'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação do euro') 
navegador.find_element('xpath',
'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# - pegar a cotação que está no google
cotacao_euro = navegador.find_element('xpath',
'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_euro)

# passo 3: Pegar a cotação do ouro
navegador.get('https://www.melhorcambio.com/ouro-hoje')
cotacao_ouro = navegador.find_element('xpath','//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(",", ".") # para substituir a virgula pelo ponto
print(cotacao_ouro)

navegador.quit()

5.1151
5.5571
316.75


### Atualizar a nossa base de preços com as novas cotações

- Importando a base de dados

In [34]:
# passo 4: Atualizar a base de preços(atualizando o preço de compra e o de venda)
import pandas as pd

tabela = pd.read_excel('Produtos.xlsx')
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [35]:
# atualizar coluna Cotação
# (Editar a coluna Cotção, onde a coluna Moeda = Dólar)
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)

tabela.loc[tabela["Moeda"] == "Euro", "Cotação"]  = float(cotacao_euro)

tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"]  = float(cotacao_ouro)

# atualizar coluna Preço de Compra = Preço Original * Cotação
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

# atualizar coluna Preço de Venda = Preço de Compra * Margem
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.1151,5115.048849,1.40,7161.068389
1,Carro Renault,4500.00,Euro,5.5571,25006.950000,2.00,50013.900000
2,Notebook Dell,899.99,Dólar,5.1151,4603.538849,1.70,7826.016043
3,IPhone,799.00,Dólar,5.1151,4086.964900,1.70,6947.840330
4,Carro Fiat,3000.00,Euro,5.5571,16671.300000,1.90,31675.470000
5,Celular Xiaomi,480.48,Dólar,5.1151,2457.703248,2.00,4915.406496
6,Joia 20g,20.00,Ouro,316.7500,6335.000000,1.15,7285.250000


### Exportar a nova base de preços atualizada

In [36]:
# passo 5: Exportar a base de preços atualizado
tabela.to_excel("Produtos Novo.xlsx", index=False)